# Converter

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json
from converter import convert_func
DATA_FOLDER = "./data/"
CAM_NAME = "camera3"

# DATA_FOLDER = "./data2/"
# CAM_NAME = "camera2"

# DATA_FOLDER = "./data0/"
# CAM_NAME = "camera0"

# DATA_FOLDER = "./data1/"
# CAM_NAME = "camera1"

DATA_FOLDER = "./data_xu/"
CAM_NAME = "camera"

%matplotlib inline

# Load groundtruth noise

In [ ]:
video_fname = "./turbine_testv3_0010-0050_share.mp4"
cap = cv2.VideoCapture(video_fname)
cap.isOpened()

In [ ]:
wo_noise_imgs = []
while True:
    ret, img = cap.read()
    if not ret:
        break
    wo_noise_imgs.append(img)

In [ ]:
for it in wo_noise_imgs:
    cv2.imshow('img', it)
    key = cv2.waitKey(300)
    if key == 27:
        break
cv2.destroyAllWindows()

In [ ]:
plt.imshow(wo_noise_imgs[0][:,:,::-1])

# Load optim functio

In [ ]:
# Load optim function
# json_fname = f"convert_params_{CAM_NAME}.json"
json_fname = f"convert_params_camera3.json"
with open(json_fname) as f:
    params = json.load(f)
    
original_fps = 12
video_fps = 24
print(f"origial fps: {original_fps}")
print(f"video fps: {video_fps}")
print("Adjust max_val")
params["max_val"] = params["max_val"] * original_fps / video_fps 
print(params)

# Generate noise image

In [ ]:
geant4_deposits = []
for i in range(1000):
    try:
        deposit = np.load(f"{DATA_FOLDER}/geant4_gamma_noise_{i}.npy")
        geant4_deposits.append(deposit)
    except:
        print("Finish loading: Total", i)
        break

In [ ]:
from converter import normalize_image, jpeg_block_noise
def convert_func_color(geant4_deposit, wo_noise, **kwargs):
    #     print('kwargs: ', kwargs)
    geant4_noise = geant4_deposit.copy()
    geant4_noise = cv2.GaussianBlur(geant4_noise, (3, 3), kwargs["blur_sigma"])
    geant4_noise_wo_jpeg = normalize_image(
        geant4_noise, min_val=0, max_val=kwargs["max_val"]
    )
    geant4_noise = jpeg_block_noise(
        geant4_noise_wo_jpeg, quality=kwargs["jpeg_quality"]
    )

    if geant4_noise.dtype != np.uint8:
        geant4_noise = (255 * geant4_noise).astype(np.uint8)

    # absolute noise model
    noisy_img = wo_noise.copy()
    mask = np.any(geant4_noise[...,None] > wo_noise, axis=-1)
    noisy_img[mask] = geant4_noise[mask][..., None]

    return noisy_img

In [ ]:
# Generate gamma noise image
noisy_imgs = []
for deposit, wo_noise in zip(geant4_deposits, wo_noise_imgs):
    noisy_img = convert_func_color(deposit, wo_noise, **params)
    noisy_imgs.append(noisy_img)

In [ ]:
for it in noisy_imgs:
    cv2.imshow('img', it)
    key = cv2.waitKey(100)
    if key == 27:
        break
cv2.destroyAllWindows()

In [ ]:
#############################
## Xu paper until here
#############################

In [ ]:
from scipy.stats import binom

# Salt-and-pepper
json_fname = f"convert_params_{CAM_NAME}_salt_pepper.json"
with open(json_fname) as f:
    sp_params = json.load(f)
    
def add_salt_and_pepper(wo_noise, p=0.1):
    s_p_noisy = wo_noise.copy()
    salt_pepper = binom.rvs(n=1, p=p, loc=0, size=img_size)
    s_p_noisy[salt_pepper==1] = 255
    return s_p_noisy

img_size = wo_noise.shape
p = 0.01
salt_pepper = binom.rvs(n=1, p=p, loc=0, size=img_size)
plt.imshow(salt_pepper)

In [ ]:
# Visualize gamma images
from itertools import cycle

# Save as video
cap_frames = {}
cap_frames["gt"] = []
cap_frames["proposed"] = []
cap_frames["salt"] = []

i = 0
while True:
    cv2.imshow("img wo noise", wo_noise)
    noisy = noisy_imgs[i%len(noisy_imgs)]
    cv2.imshow("noisy img", noisy)
    cap_frames["proposed"].append(noisy)
    
    gt = gt_noises[i%len(gt_noises)]
    cap_frames["gt"].append(gt)
    cv2.imshow("gt noise", gt)
    
    s_p_noisy = add_salt_and_pepper(wo_noise, p=sp_params['prob'])
    cv2.imshow("Salt-and-pepper noise", s_p_noisy)
    cap_frames["salt"].append(s_p_noisy)

    key = cv2.waitKey(40)
    if key == 27:
        break
    i += 1
    if i==(12*10):
        break
cv2.destroyAllWindows()

In [ ]:
len(cap_frames["proposed"])

In [ ]:
# from os.path import join
# VIDEO_FOLDER = "presen_video"
# frame_rate = 12.0
# size = (256, 256)

# for key, imgs in cap_frames.items():
# #     video_fname = join(VIDEO_FOLDER, f"{CAM_NAME}_{key}.mp4")
#     video_fname = join(VIDEO_FOLDER, f"{CAM_NAME}_{key}.avi")
#     print(video_fname)
# #     fourcc = cv2.VideoWriter_fourcc(*"mp4v")
#     fourcc = cv2.VideoWriter_fourcc(*'XVID')
#     writer = cv2.VideoWriter(video_fname, fourcc, frame_rate, size) # ライター作成
#     for img in imgs:
#         writer.write(img)
#     writer.release()

In [ ]:
import os
from os.path import join
VIDEO_FOLDER = "presen_video"+CAM_NAME[-1]
os.mkdir(VIDEO_FOLDER)

frame_rate = 12.0
size = (256, 256)

for key, imgs in cap_frames.items():
    for i, img in enumerate(imgs):
        video_fname = join(VIDEO_FOLDER, f"{CAM_NAME}_{key}_{i:03}.jpg")
        if (i%60)==0:
            print(video_fname)
        cv2.imwrite(video_fname, img)

    msg = f"ffmpeg -r 12 -f image2 -s 256x256 -i {join(VIDEO_FOLDER, CAM_NAME)}_{key}_%03d.jpg -vcodec libx264 -crf 10 -pix_fmt yuv420p {join(VIDEO_FOLDER, CAM_NAME)}_{key}.mp4"
    print(msg)
    !$msg

In [ ]:
video_fname = 'presen_video/'

In [ ]:
# wo_noise = cv2.imread(f"{DATA_FOLDER}/image_wo_noise.png", 0)
# s_p_imgs = [add_salt_and_pepper(wo_noise, p=sp_params["prob"]) for _ in range(5)]
# PAPER_FOLDER = f"./{CAM_NAME}_noise_paper/"
# for i, img in enumerate(s_p_imgs):
#     cv2.imwrite(f"{PAPER_FOLDER}/sp_noise_img_{i}.png", img)
# print(sp_params)

# Metrics

In [ ]:
CAM_NAME

In [ ]:
PAPER_FOLDER = f"./{CAM_NAME}_noise_paper/"
print(PAPER_FOLDER)
proposed_imgs = [cv2.imread(f"{PAPER_FOLDER}/generated_noise_img_{i}.png", 0) for i in range(5)]
s_p_imgs = [cv2.imread(f"{PAPER_FOLDER}/sp_noise_img_{i}.png", 0) for i in range(5)]
gt_noisy_imgs = [cv2.imread(f"{PAPER_FOLDER}/gt_noise_img_{i}.png", 0) for i in range(5)]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import img_as_float
from skimage.metrics import structural_similarity as ssim

img = img_as_float(gt_noisy_imgs[0])
img_proposed = img_as_float(proposed_imgs[0])
img_sp = img_as_float(s_p_imgs[0])

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10, 4),
                         sharex=True, sharey=True)
ax = axes.ravel()

ssim_proposed = ssim(img, img_proposed)
ssim_sp = ssim(img, img_sp)

label = 'SSIM: {:.2f}'

ax[0].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
ax[0].set_title('Groudtruth image')

ax[1].imshow(img_proposed, cmap=plt.cm.gray, vmin=0, vmax=1)
ax[1].set_xlabel(label.format(ssim_proposed))
ax[1].set_title('Proposed noisy image')

ax[2].imshow(img_sp, cmap=plt.cm.gray, vmin=0, vmax=1)
ax[2].set_xlabel(label.format(ssim_sp))
ax[2].set_title('Salt and pepper noisy image')

plt.tight_layout()
plt.savefig(f"{PAPER_FOLDER}/plot_sim.pdf")
plt.show()


In [ ]:
msg = '[{}] SSIM: {:.3f}±{:.3f}'
# Proposed method score
a = gt_noisy_imgs
b = proposed_imgs
proposed_scores = []
for gt, img in ((x,y) for x in a for y in b):
    gt = img_as_float(gt)
    img = img_as_float(img)
    score = ssim(gt, img)
    proposed_scores.append(score)
print(msg.format("proposed", np.average(proposed_scores), np.std(proposed_scores)))

# Salt-and-pepper method score
a = gt_noisy_imgs
b = s_p_imgs
sp_scores = []
for gt, img in ((x,y) for x in a for y in b):
    gt = img_as_float(gt)
    img = img_as_float(img)
    score = ssim(gt, img)
    sp_scores.append(score)
    
print(msg.format("salt-and-pepper", np.average(sp_scores), np.std(sp_scores)))

In [ ]:
with open(f"{PAPER_FOLDER}/ssim_score.txt", 'w') as f:
    msgs = [msg.format("proposed", np.average(proposed_scores), np.std(proposed_scores)),
            msg.format("salt-and-pepper", np.average(sp_scores), np.std(sp_scores))]
    f.writelines("\n".join(msgs))